In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from pathlib import Path

In [2]:
# Data Preparation
data_dir = '../artifacts/data_ingestion/Chest-CT-Scan-data'
img_size = (224, 224)
batch_size = 32

# train_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(data_dir,
                                            target_size = img_size,
                                            batch_size = batch_size,
                                            class_mode = 'binary',
                                            subset = "training")

test_generator = datagen.flow_from_directory(data_dir,
                                            target_size = img_size,
                                            batch_size = batch_size,
                                            class_mode = 'binary',
                                            subset = "validation")

# Model Definition
base_model = VGG16(weights = 'imagenet', include_top=False, input_shape=(224,224,3))
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = "sigmoid"))

# Freeze the convolutional base
base_model.trainable = False

Found 275 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


In [3]:
# Compile the model
model.compile(loss = "binary_crossentropy",
              optimizer = optimizers.RMSprop(learning_rate=1e-4),
              metrics = ["accuracy"])

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
# Define Early Stopping callback
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience =2, restore_best_weights = True)

In [5]:
# Model Training
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples / train_generator.batch_size,
    epochs = 5,
    validation_data = test_generator,
    validation_steps = test_generator.samples / test_generator.batch_size,
    verbose = 1,
    callbacks = [early_stopping]
)

# Model Evaluation
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
print("Test Accuracy:", test_acc)

# Prediction
predictions = model.predict(test_generator)

Epoch 1/5
8/8 [==============================] - 79s 9s/step - loss: 0.8223 - accuracy: 0.6836 - val_loss: 0.4456 - val_accuracy: 0.7206
Epoch 2/5
8/8 [==============================] - 74s 9s/step - loss: 0.2916 - accuracy: 0.8691 - val_loss: 0.0705 - val_accuracy: 1.0000
Epoch 3/5
8/8 [==========================>...] - ETA: 4s - loss: 0.1274 - accuracy: 0.9671 

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: '../artifacts/data_ingestion/Chest-CT-Scan-data\\adenocarcinoma\\000050 (3).png'
Traceback (most recent call last):

  File "c:\Users\Satyam\.conda\envs\ml_pipeline\lib\site-packages\tensorflow\python\ops\script_ops.py", line 267, in __call__
    ret = func(*args)

  File "c:\Users\Satyam\.conda\envs\ml_pipeline\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\Satyam\.conda\envs\ml_pipeline\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\Satyam\.conda\envs\ml_pipeline\lib\site-packages\keras\engine\data_adapter.py", line 902, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\Satyam\.conda\envs\ml_pipeline\lib\site-packages\keras\engine\data_adapter.py", line 1049,

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Make Predictions
predictions = model.predict(test_generator)
predictions
predicted_classes = np.round(predictions).flatten()

# Get true labels
true_labels = test_generator.classes

# Calculate Metrics
acc_score = accuracy_score(true_labels, predicted_classes)
conf_matrix = confusion_matrix(true_labels, predicted_classes)
class_report = classification_report(true_labels, predicted_classes)

# Print Metrics
print("Accuracy:", acc_score)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

3/3 [==============================] - 12s 3s/step
Accuracy: 0.6470588235294118
Confusion Matrix:
[[27 12]
 [12 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.69      0.69        39
           1       0.59      0.59      0.59        29

    accuracy                           0.65        68
   macro avg       0.64      0.64      0.64        68
weighted avg       0.65      0.65      0.65        68



In [ ]:
n=0
for true, pred in zip(true_labels, predicted_classes):
    if n < 10:
        n+=1
        print(true, "<<<<True Label************ Pred Label>>>>",  pred)

0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 1.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0
0 <<<<True Label************ Pred Label>>>> 0.0


In [ ]:
print(train_generator.class_indices)

{'adenocarcinoma': 0, 'normal': 1}
